In [1]:
! pip uninstall spacy -y
!pip install spacy

Found existing installation: spacy 3.4.2
Uninstalling spacy-3.4.2:
  Successfully uninstalled spacy-3.4.2
  Using cached spacy-3.4.2-cp38-cp38-macosx_10_9_x86_64.whl (6.6 MB)


In [2]:
! python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 6.3 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [305]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.stem.lancaster import LancasterStemmer
from nltk.tag import pos_tag
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem.porter import *
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import PCA
from spacy.lang.en.stop_words import STOP_WORDS

In [306]:
!pip install gensim

In [307]:
pip install -U gensim

Note: you may need to restart the kernel to use updated packages.


In [308]:
import gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [309]:
nltk.download('maxent_ne_chunker')
from nltk.chunk import ne_chunk
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/riwasabri/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/riwasabri/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [310]:
!conda install gensim -y

Solving environment: done

# All requested packages already installed.

Retrieving notices: ...working... done


In [311]:
with open('nightmares_tweets.pickle', 'rb') as handle:
    df = pickle.load(handle)

### Pre-Processing

In [312]:
#Removing unnecessary columns

In [313]:
df.shape

(6168, 28)

In [314]:
sum([len(d.split(' ')) for d in df.content]) 

180575

In [315]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6168 entries, 0 to 6167
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   _type             6168 non-null   object             
 1   url               6168 non-null   object             
 2   date              6168 non-null   datetime64[ns, UTC]
 3   content           6168 non-null   object             
 4   renderedContent   6168 non-null   object             
 5   id                6168 non-null   int64              
 6   user              6168 non-null   object             
 7   replyCount        6168 non-null   int64              
 8   retweetCount      6168 non-null   int64              
 9   likeCount         6168 non-null   int64              
 10  quoteCount        6168 non-null   int64              
 11  conversationId    6168 non-null   int64              
 12  lang              6168 non-null   object             
 13  sou

In [316]:
df.columns

Index(['_type', 'url', 'date', 'content', 'renderedContent', 'id', 'user',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel',
       'outlinks', 'tcooutlinks', 'media', 'retweetedTweet', 'quotedTweet',
       'inReplyToTweetId', 'inReplyToUser', 'mentionedUsers', 'coordinates',
       'place', 'hashtags', 'cashtags'],
      dtype='object')

In [322]:
for i in df.content:
    print(i)

Last Night I had a nightmare about a chain chomp from super mario 64 and I dont know why but I woke up sweating and I was like wth its just a chain chomp
Mario 64 personalised has ruined me
I wish I had a Gordon Ramsay in my life who could help me optimize by yelling about issues I’m aware of. 

“Stop watching Kitchen Nightmare clips on YouTube”. I slowly internalize Gordon, “Hurry the fuck up. Internalize and get to the next thing.”
@eponymous I think I had a nightmare about this table last night.
Corona got me so nervous, I had a nightmare that I was at a gathering,
NO MASK, and suddenly started panicking about how many ppl I was around. 😭😭 randomly @BombshellChanel you were there too! Mask less dab!!!! 😂😂
I’ve been trading so much past few days, I think I had a nightmare about buying a card for 100k over the actual price 😭
I think i pulled her to bed last night by hysterically crying...I had a nightmare about her leaving me &amp; my parents telling me my work was worthless so
@mharr

In [323]:
df.drop(columns=['date','_type','renderedContent', 'url','user','replyCount','retweetCount','likeCount','quoteCount',\
                           'conversationId','source','sourceUrl','sourceLabel','inReplyToUser','coordinates','place',
                            'retweetedTweet','media'],inplace=True)

### I am leaving a few unnecessary columns here that are going to help with cleaning out the text

In [324]:
#Removing 'quotedTweet' and 'retweetedTweet' and 'inReplyToTweetId' and 'cashtags' are not none to leave only original 
#and non original tweets 

In [325]:
mask=(df.inReplyToTweetId.isna()) & (df.lang=='en') & (df.quotedTweet.isna()) & (df.cashtags.isna()) & (df.cashtags.isna())
df=df[mask].drop(columns=['inReplyToTweetId','lang','quotedTweet','cashtags'])

In [326]:
df[20:30]

,content,id,outlinks,tcooutlinks,mentionedUsers,hashtags
31,"I had a terrifying nightmare once, whilst I wa...",1334464079860289538,None,None,None,None
32,last night I had a nightmare about my ex being...,1334463148754149376,None,None,None,None
33,i had a nightmare about me coming out to my da...,1334461834347372544,None,None,None,None
34,I had a nightmare about demons chasing me in h...,1334459230666043393,None,None,None,None
35,I had a nightmare about a fatal accident. Luck...,1334456037378756609,None,None,None,None
39,"Today I came out as non-binary, tonight I had ...",1334411825388064769,None,None,None,None
40,"MORNING, I had a dream/nightmare about a demon...",1334400707609419778,None,None,None,None
41,khh I HAD A NIGHTMARE ABOUT MY OLD DAYS,1334396375753101312,None,None,None,None
44,I had a nightmare me about going to the dentis...,1334390211447582723,None,None,None,None
48,i had a nightmare about moving to rural puncak...,1334355563820380160,None,None,None,None


### Pre-processing of text

In [361]:
def pre_process(text):
    #making text lowercase and deleting '\n','&amp'
    text= re.sub('(\n|&amp)',' ',text.lower())
    # removing hyperlinks
    text= re.sub('(http|ftp|https|www):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])','',text)
    #removing mentions
    text=re.sub('@[A-Za-z0-9-_]+','',text)
    text=re.sub('@[ ]+[A-Za-z0-9-_]+','',text)
    #removing hashtags
    text=re.sub('#[A-Za-z0-9-_]+','',text)
    #removing emojis and other character that are not alphanumerical
    text=re.sub("[^a-z\s\(\-:\)\\\/\];='#]",'',text)
    #words
    text=re.sub('night[A-Za-z0-9-_]+','',text)
    text=re.sub('sleep[A-Za-z0-9-_]+','',text)
    text=re.sub('[A-Za-z0-9-_]sleep','',text)
    text=re.sub('[A-Za-z0-9-_]night','',text)
    text=re.sub('dream[A-Za-z0-9-_]+','',text)
    text=re.sub('wake[A-Za-z0-9-_]+','',text)
    text=re.sub('[A-Za-z0-9-_]wake','',text)
    text=re.sub('think[A-Za-z0-9-_]+','',text)
    text=re.sub('wake','',text)
    text=re.sub('lmao','',text)
    text=re.sub('[A-Za-z0-9-_]wake','',text)
    text=re.sub('feel[A-Za-z0-9-_]','',text)
    text=re.sub('day[A-Za-z0-9-_]','',text)
    text=re.sub('today','',text)
    text=re.sub('terrible','',text)
    text=re.sub('try','',text)
    text=re.sub('hour','',text)
    text=re.sub('watch','',text)
    text=re.sub('come','',text)
    text=re.sub('say','',text)
    text=re.sub('said','',text)
    text=re.sub('year','',text)
    return text

In [329]:
df['original']=df.content

In [330]:
df['content']=df.content.apply(lambda x: pre_process(x))

In [331]:
df.drop(columns=['id','outlinks','tcooutlinks','mentionedUsers','hashtags'],inplace=True)

In [332]:
df.reset_index(inplace=True)

### More text Processing: Lemmatization + Stopwords removal 

In [333]:
import spacy

In [334]:
nlp = spacy.load('en_core_web_sm')
stemmer= PorterStemmer()

In [335]:
def remove_common(sentence):
    list2=['night','dream', 'think', 'wake', 'like', 'sleep', 'remember', 'feel', 'time', 'bad','know', 'day', 'good', 'thing', 'tell', 'talk','want', 'happen','think','ago','morning','woke','lol','wtf']
    new_word=[]
    for i in sentence.split(' '):
        if i in list2:
            new_word.append('')
        else:
            new_word.append(i)
    return ' '.join(new_word)

In [336]:
df['content']=df.content.apply(lambda x:remove_common(x))

In [337]:
for doc in df['content']:
    if 'woke' in doc:
        print (doc)

i finally fell  early only to be woken up because i had a  about a gate and a driveway (i can't  the rest)(i'm assuming there's more lol)
as i was falling  last  i had a really  tweet idea i didn't write it down bc i was certain i would  it in the  the  came i awoke crying from  and completely forgot about the tweet until right now i still do not  it
at  am i had a lucid  about my   old getting lost in a dollar tree while i spoke with my former acting coach  ; drinking fabuloso -i found her palevomiting and convulsingwoke up nauseated  ; emotional couldn't go back to  kept checking in on her
i had a ng it was about to turn into a  something just didnt  right but because i couldnt move i couldnt look around and do anything all i could do was listen and pretend to be  because the people in the room noticed my presence and had now woken
election has me so stressed and anxious that i had a  about herman cain temporarily taking over trumps role in the white house   i have never woken up so 

In [338]:
docs = list(nlp.pipe(df.content))

In [339]:
nlp.Defaults.stop_words |= {"nightmare","nightmares",'night','dream','sleep','asleep','rembember','fucking','fuck','remembered'}

In [340]:
docs_clean = [[w.lemma_ for w in doc if (not w.is_stop and not w.is_punct and not w.like_num and len(w)>2 and w.pos_ != 'ADJ')\
               or (w.lemma_=='not')] for doc in docs]
df['content_clean'] = docs_clean

In [341]:
docs_list_clean = [' '.join(map(str,doc)) for doc in docs_clean]

## Topic Modeling: Testing out different dimensionality reduction techniques

### With tf-idf vectorizer

In [342]:
tv = TfidfVectorizer(stop_words='english',min_df=10,max_df=0.80)
X = tv.fit_transform(docs_list_clean)

df_X = pd.DataFrame(X.toarray(), columns=tv.get_feature_names())

In [485]:
docs_list_clean[1]

'corona get gathering mask suddenly start panic ppl randomly mask dab'

In [343]:
#function that returns top topics 

In [344]:
def display_topics(model, feature_names, no_top_words, topic_names = None): 
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix + 1)
        else:
            print("\nTopic: ", topic_names[ix])
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
    print("\n")
    return model, feature_names, no_top_words

'Corona got me so nervous, I had a nightmare that I was at a gathering,\nNO MASK, and suddenly started panicking about how many ppl I was around. 😭😭 randomly @BombshellChanel you were there too! Mask less dab!!!! 😂😂'

In [345]:
#PCA

In [362]:
pca = PCA(n_components=10)
pca.fit(df_X)
display_topics(pca, tv.get_feature_names(), 5)


Topic  1
work, job, stress, customer, sleep

Topic  2
play, junko, enoshima, dangle, rope

Topic  3
school, play, enoshima, junko, rope

Topic  4
mask, wear, people, forget, store

Topic  5
stop, forget, class, election, trump

Topic  6
friend, hate, stop, happen, people

Topic  7
house, stop, forget, kid, need

Topic  8
forget, friend, hate, happen, class

Topic  9
people, election, shit, forget, trump

Topic  10
guy, friend, love, election, need




(PCA(n_components=10),
 ['absolutely',
  'abt',
  'abuse',
  'accident',
  'accidentally',
  'account',
  'actually',
  'affect',
  'air',
  'alarm',
  'allow',
  'america',
  'animal',
  'answer',
  'anxiety',
  'anymore',
  'anyways',
  'apartment',
  'apocalypse',
  'apparently',
  'argue',
  'arm',
  'art',
  'ask',
  'ass',
  'assignment',
  'attack',
  'away',
  'baby',
  'ballot',
  'bar',
  'barely',
  'basically',
  'bathroom',
  'bear',
  'beat',
  'bed',
  'believe',
  'biden',
  'bit',
  'bitch',
  'bite',
  'blame',
  'blow',
  'body',
  'book',
  'boss',
  'boy',
  'boyfriend',
  'brain',
  'break',
  'bring',
  'bro',
  'brother',
  'bruh',
  'bts',
  'bug',
  'bully',
  'bunch',
  'burn',
  'buy',
  'calm',
  'cancel',
  'car',
  'care',
  'case',
  'cat',
  'catch',
  'cause',
  'change',
  'character',
  'chase',
  'check',
  'cheese',
  'child',
  'childhood',
  'chill',
  'christmas',
  'city',
  'class',
  'close',
  'clown',
  'college',
  'come',
  'completely',


In [347]:
#LSA 

In [348]:
lsa = TruncatedSVD(n_components=5)
lsa.fit(df_X)
display_topics(lsa, tv.get_feature_names(), 5)


Topic  1
work, school, people, mask, friend

Topic  2
work, job, customer, office, honestly

Topic  3
play, junko, enoshima, dangle, rope

Topic  4
school, class, work, week, exam

Topic  5
mask, forget, wear, people, store




(TruncatedSVD(n_components=5),
 ['absolutely',
  'abt',
  'abuse',
  'accident',
  'accidentally',
  'account',
  'actually',
  'affect',
  'air',
  'alarm',
  'allow',
  'america',
  'animal',
  'answer',
  'anxiety',
  'anymore',
  'anyways',
  'apartment',
  'apocalypse',
  'apparently',
  'argue',
  'arm',
  'art',
  'ask',
  'ass',
  'assignment',
  'attack',
  'away',
  'baby',
  'ballot',
  'bar',
  'barely',
  'basically',
  'bathroom',
  'bear',
  'beat',
  'bed',
  'believe',
  'biden',
  'bit',
  'bitch',
  'bite',
  'blame',
  'blow',
  'body',
  'book',
  'boss',
  'boy',
  'boyfriend',
  'brain',
  'break',
  'bring',
  'bro',
  'brother',
  'bruh',
  'bts',
  'bug',
  'bully',
  'bunch',
  'burn',
  'buy',
  'calm',
  'cancel',
  'car',
  'care',
  'case',
  'cat',
  'catch',
  'cause',
  'change',
  'character',
  'chase',
  'check',
  'cheese',
  'child',
  'childhood',
  'chill',
  'christmas',
  'city',
  'class',
  'close',
  'clown',
  'college',
  'come',
  'compl

In [349]:
#NMF

In [350]:
nmf = NMF(5)
nmf.fit(df_X)
display_topics(nmf, tv.get_feature_names(), 10)


Topic  1
friend, house, shit, people, happen, hate, guy, stop, tell, start

Topic  2
work, job, stress, customer, sleep, nap, wake, office, leave, honestly

Topic  3
play, enoshima, junko, dangle, rope, game, man, song, world, relate

Topic  4
school, class, start, exam, teacher, week, miss, test, math, bully

Topic  5
mask, forget, wear, people, store, crowd, panic, class, face, pandemic




/Users/riwasabri/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


(NMF(n_components=5),
 ['absolutely',
  'abt',
  'abuse',
  'accident',
  'accidentally',
  'account',
  'actually',
  'affect',
  'air',
  'alarm',
  'allow',
  'america',
  'animal',
  'answer',
  'anxiety',
  'anymore',
  'anyways',
  'apartment',
  'apocalypse',
  'apparently',
  'argue',
  'arm',
  'art',
  'ask',
  'ass',
  'assignment',
  'attack',
  'away',
  'baby',
  'ballot',
  'bar',
  'barely',
  'basically',
  'bathroom',
  'bear',
  'beat',
  'bed',
  'believe',
  'biden',
  'bit',
  'bitch',
  'bite',
  'blame',
  'blow',
  'body',
  'book',
  'boss',
  'boy',
  'boyfriend',
  'brain',
  'break',
  'bring',
  'bro',
  'brother',
  'bruh',
  'bts',
  'bug',
  'bully',
  'bunch',
  'burn',
  'buy',
  'calm',
  'cancel',
  'car',
  'care',
  'case',
  'cat',
  'catch',
  'cause',
  'change',
  'character',
  'chase',
  'check',
  'cheese',
  'child',
  'childhood',
  'chill',
  'christmas',
  'city',
  'class',
  'close',
  'clown',
  'college',
  'come',
  'completely',
 

In [351]:
#LDA

In [352]:
doc_word = tv.fit_transform(docs_list_clean).transpose()

In [353]:
pd.DataFrame(doc_word.toarray(), tv.get_feature_names())

,0,1,2,3,4,5,6,7,8,9,...,4221,4222,4223,4224,4225,4226,4227,4228,4229,4230
absolutely,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abuse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accident,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accidentally,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yesterday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [354]:
corpus = matutils.Sparse2Corpus(doc_word)

In [355]:
id2word = dict((v, k) for k, v in tv.vocabulary_.items())

In [356]:
lda = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes=100)

2022-10-26 23:11:06,373 : INFO : using symmetric alpha at 0.2
2022-10-26 23:11:06,376 : INFO : using symmetric eta at 0.2
2022-10-26 23:11:06,376 : INFO : using serial LDA version on this node
2022-10-26 23:11:06,381 : INFO : running online (multi-pass) LDA training, 5 topics, 100 passes over the supplied corpus of 4231 documents, updating model once every 2000 documents, evaluating perplexity every 4231 documents, iterating 50x with a convergence threshold of 0.001000
2022-10-26 23:11:06,391 : INFO : PROGRESS: pass 0, at document #2000/4231
2022-10-26 23:11:06,668 : INFO : merging changes from 2000 documents into a model of 4231 documents
2022-10-26 23:11:06,671 : INFO : topic #0 (0.200): 0.023*"people" + 0.013*"friend" + 0.013*"start" + 0.011*"wear" + 0.011*"play" + 0.010*"murder" + 0.010*"guy" + 0.010*"mask" + 0.010*"force" + 0.009*"head"
2022-10-26 23:11:06,671 : INFO : topic #1 (0.200): 0.018*"man" + 0.017*"die" + 0.014*"forget" + 0.014*"school" + 0.014*"lose" + 0.013*"stress" + 0

In [357]:
lda.print_topics()

2022-10-26 23:11:41,615 : INFO : topic #0 (0.200): 0.026*"people" + 0.025*"play" + 0.021*"end" + 0.019*"kill" + 0.018*"maybe" + 0.017*"bed" + 0.017*"anxiety" + 0.016*"tweet" + 0.016*"fall" + 0.016*"friend"
2022-10-26 23:11:41,616 : INFO : topic #1 (0.200): 0.034*"school" + 0.031*"man" + 0.030*"hate" + 0.025*"nap" + 0.019*"miss" + 0.018*"class" + 0.017*"cause" + 0.017*"getting" + 0.016*"lose" + 0.016*"die"
2022-10-26 23:11:41,617 : INFO : topic #2 (0.200): 0.052*"work" + 0.037*"mask" + 0.023*"kid" + 0.022*"literally" + 0.020*"wear" + 0.018*"election" + 0.017*"win" + 0.017*"house" + 0.016*"life" + 0.015*"happen"
2022-10-26 23:11:41,618 : INFO : topic #3 (0.200): 0.024*"movie" + 0.023*"home" + 0.023*"stop" + 0.022*"spider" + 0.020*"tell" + 0.019*"look" + 0.017*"yesterday" + 0.015*"turn" + 0.015*"way" + 0.013*"instead"
2022-10-26 23:11:41,619 : INFO : topic #4 (0.200): 0.032*"love" + 0.028*"eat" + 0.023*"girl" + 0.023*"shit" + 0.023*"brain" + 0.023*"sweat" + 0.022*"story" + 0.022*"write" +

[(0,
  '0.026*"people" + 0.025*"play" + 0.021*"end" + 0.019*"kill" + 0.018*"maybe" + 0.017*"bed" + 0.017*"anxiety" + 0.016*"tweet" + 0.016*"fall" + 0.016*"friend"'),
 (1,
  '0.034*"school" + 0.031*"man" + 0.030*"hate" + 0.025*"nap" + 0.019*"miss" + 0.018*"class" + 0.017*"cause" + 0.017*"getting" + 0.016*"lose" + 0.016*"die"'),
 (2,
  '0.052*"work" + 0.037*"mask" + 0.023*"kid" + 0.022*"literally" + 0.020*"wear" + 0.018*"election" + 0.017*"win" + 0.017*"house" + 0.016*"life" + 0.015*"happen"'),
 (3,
  '0.024*"movie" + 0.023*"home" + 0.023*"stop" + 0.022*"spider" + 0.020*"tell" + 0.019*"look" + 0.017*"yesterday" + 0.015*"turn" + 0.015*"way" + 0.013*"instead"'),
 (4,
  '0.032*"love" + 0.028*"eat" + 0.023*"girl" + 0.023*"shit" + 0.023*"brain" + 0.023*"sweat" + 0.022*"story" + 0.022*"write" + 0.021*"week" + 0.021*"place"')]

### With count vectorizer

In [358]:
cv = CountVectorizer(ngram_range=(1,2),stop_words='english',min_df=10,max_df=0.80)
X2 = cv.fit_transform(docs_list_clean)

df_X2 = pd.DataFrame(X2.toarray(), columns=cv.get_feature_names())

In [359]:
# PCA

In [360]:
pca = PCA(n_components=5)
pca.fit(df_X2)
display_topics(pca, cv.get_feature_names(), 10)


Topic  1
play, rope, dangle, junko, junko enoshima, dangle rope, enoshima, enoshima play, play dangle, game

Topic  2
work, mask, wear, wear mask, job, people, play, office, actually, stress

Topic  3
mask, people, wear, wear mask, forget, school, start, friend, store, crowd

Topic  4
school, class, forget, start, exam, math, teacher, test, miss, graduate

Topic  5
people, friend, school, house, leave, tell, kill, start, guy, talk




(PCA(n_components=5),
 ['absolutely',
  'abt',
  'abuse',
  'accident',
  'accidentally',
  'account',
  'actually',
  'affect',
  'air',
  'alarm',
  'allow',
  'america',
  'animal',
  'answer',
  'anxiety',
  'anymore',
  'anyways',
  'apartment',
  'apocalypse',
  'apparently',
  'argue',
  'arm',
  'art',
  'ask',
  'ass',
  'assignment',
  'attack',
  'away',
  'baby',
  'ballot',
  'bar',
  'barely',
  'basically',
  'bathroom',
  'bear',
  'beat',
  'bed',
  'believe',
  'biden',
  'bit',
  'bitch',
  'bite',
  'blame',
  'blow',
  'body',
  'book',
  'boss',
  'boy',
  'boyfriend',
  'brain',
  'break',
  'bring',
  'bro',
  'brother',
  'bruh',
  'bts',
  'bug',
  'bully',
  'bunch',
  'burn',
  'buy',
  'calm',
  'cancel',
  'car',
  'care',
  'case',
  'cat',
  'catch',
  'cause',
  'change',
  'character',
  'chase',
  'check',
  'cheese',
  'child',
  'childhood',
  'chill',
  'christmas',
  'city',
  'class',
  'close',
  'clown',
  'college',
  'come',
  'completely',
 

In [240]:
#LSA

In [241]:
lsa = TruncatedSVD(n_components=5)
lsa.fit(df_X2)
display_topics(lsa, cv.get_feature_names(), 10)


Topic  1
work, people, mask, start, school, friend, house, forget, fuck, leave

Topic  2
play, rope, dangle, junko, enoshima, junko enoshima, enoshima play, play dangle, dangle rope, game

Topic  3
work, job, office, week, customer, coworker, stress, boss, early, return

Topic  4
mask, wear, wear mask, people, forget, work, store, crowd, pandemic, group

Topic  5
school, class, forget, start, exam, math, teacher, test, graduate, miss




(TruncatedSVD(n_components=5),
 ['absolutely',
  'abt',
  'abuse',
  'accident',
  'accidentally',
  'account',
  'actually',
  'affect',
  'air',
  'alarm',
  'allow',
  'america',
  'animal',
  'answer',
  'anxiety',
  'anymore',
  'anyways',
  'apartment',
  'apocalypse',
  'apparently',
  'argue',
  'arm',
  'ask',
  'ass',
  'assignment',
  'attack',
  'away',
  'baby',
  'ballot',
  'bar',
  'barely',
  'basically',
  'bathroom',
  'bear',
  'beat',
  'bed',
  'believe',
  'biden',
  'bit',
  'bitch',
  'bite',
  'blame',
  'blow',
  'body',
  'book',
  'boss',
  'boy',
  'boyfriend',
  'brain',
  'break',
  'bring',
  'bro',
  'brother',
  'bruh',
  'bts',
  'bug',
  'bully',
  'bunch',
  'burn',
  'buy',
  'calm',
  'cancel',
  'car',
  'care',
  'case',
  'cat',
  'catch',
  'cause',
  'change',
  'character',
  'chase',
  'check',
  'cheese',
  'child',
  'childhood',
  'chill',
  'christmas',
  'city',
  'class',
  'close',
  'clown',
  'college',
  'come',
  'completely',
 

In [242]:
#NMF

In [243]:
nmf.fit(df_X2)
display_topics(nmf, cv.get_feature_names(), 10)


Topic  1
friend, house, people, fuck, start, tell, leave, shit, life, happen

Topic  2
play, rope, dangle, junko, enoshima play, junko enoshima, enoshima, play dangle, dangle rope, game

Topic  3
work, job, office, actually, week, stress, ing, right, scream, come

Topic  4
mask, wear, wear mask, people, forget, store, crowd, pandemic, realize, care

Topic  5
school, class, forget, exam, start, math, miss, teacher, fail, test




/Users/riwasabri/opt/anaconda3/lib/python3.9/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


(NMF(n_components=5),
 ['absolutely',
  'abt',
  'abuse',
  'accident',
  'accidentally',
  'account',
  'actually',
  'affect',
  'air',
  'alarm',
  'allow',
  'america',
  'animal',
  'answer',
  'anxiety',
  'anymore',
  'anyways',
  'apartment',
  'apocalypse',
  'apparently',
  'argue',
  'arm',
  'ask',
  'ass',
  'assignment',
  'attack',
  'away',
  'baby',
  'ballot',
  'bar',
  'barely',
  'basically',
  'bathroom',
  'bear',
  'beat',
  'bed',
  'believe',
  'biden',
  'bit',
  'bitch',
  'bite',
  'blame',
  'blow',
  'body',
  'book',
  'boss',
  'boy',
  'boyfriend',
  'brain',
  'break',
  'bring',
  'bro',
  'brother',
  'bruh',
  'bts',
  'bug',
  'bully',
  'bunch',
  'burn',
  'buy',
  'calm',
  'cancel',
  'car',
  'care',
  'case',
  'cat',
  'catch',
  'cause',
  'change',
  'character',
  'chase',
  'check',
  'cheese',
  'child',
  'childhood',
  'chill',
  'christmas',
  'city',
  'class',
  'close',
  'clown',
  'college',
  'come',
  'completely',
  'concert

In [244]:
#LDA

In [245]:
doc_word_2= cv.fit_transform(docs_list_clean).transpose()

In [246]:
pd.DataFrame(doc_word_2.toarray(), cv.get_feature_names())

,0,1,2,3,4,5,6,7,8,9,...,4221,4222,4223,4224,4225,4226,4227,4228,4229,4230
absolutely,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abt,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
abuse,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
accident,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
accidentally,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
yesterday,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zombie apocalypse,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [247]:
corpus = matutils.Sparse2Corpus(doc_word_2)

In [248]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [249]:
lda2 = models.LdaModel(corpus=corpus, num_topics=5, id2word=id2word, passes=10,random_state=42)

2022-10-26 12:29:28,912 : INFO : using symmetric alpha at 0.2
2022-10-26 12:29:28,913 : INFO : using symmetric eta at 0.2
2022-10-26 12:29:28,914 : INFO : using serial LDA version on this node
2022-10-26 12:29:28,915 : INFO : running online (multi-pass) LDA training, 5 topics, 10 passes over the supplied corpus of 4231 documents, updating model once every 2000 documents, evaluating perplexity every 4231 documents, iterating 50x with a convergence threshold of 0.001000
2022-10-26 12:29:28,921 : INFO : PROGRESS: pass 0, at document #2000/4231
2022-10-26 12:29:29,267 : INFO : merging changes from 2000 documents into a model of 4231 documents
2022-10-26 12:29:29,268 : INFO : topic #0 (0.200): 0.017*"work" + 0.014*"remember" + 0.014*"fuck" + 0.013*"happen" + 0.012*"person" + 0.012*"life" + 0.011*"movie" + 0.011*"guy" + 0.010*"head" + 0.009*"feel"
2022-10-26 12:29:29,268 : INFO : topic #1 (0.200): 0.032*"work" + 0.019*"stop" + 0.017*"fall" + 0.015*"start" + 0.014*"forget" + 0.011*"brain" + 0

In [250]:
lda2.print_topics()

2022-10-26 12:29:33,427 : INFO : topic #0 (0.200): 0.058*"fuck" + 0.040*"play" + 0.034*"brain" + 0.033*"movie" + 0.032*"remember" + 0.024*"shake" + 0.020*"time" + 0.019*"happen" + 0.019*"girl" + 0.016*"early"
2022-10-26 12:29:33,427 : INFO : topic #1 (0.200): 0.065*"work" + 0.029*"house" + 0.028*"tell" + 0.022*"mean" + 0.021*"shit" + 0.021*"stop" + 0.020*"kid" + 0.016*"eye" + 0.015*"way" + 0.014*"live"
2022-10-26 12:29:33,428 : INFO : topic #2 (0.200): 0.033*"love" + 0.027*"wake" + 0.025*"think" + 0.024*"face" + 0.021*"write" + 0.021*"dog" + 0.020*"yesterday" + 0.017*"read" + 0.016*"instead" + 0.016*"start"
2022-10-26 12:29:33,428 : INFO : topic #3 (0.200): 0.040*"mask" + 0.037*"school" + 0.028*"people" + 0.023*"spider" + 0.021*"attack" + 0.020*"wear" + 0.019*"panic" + 0.017*"forget" + 0.017*"point" + 0.016*"week"
2022-10-26 12:29:33,429 : INFO : topic #4 (0.200): 0.025*"turn" + 0.025*"hate" + 0.021*"home" + 0.018*"ass" + 0.018*"ing" + 0.018*"lot" + 0.018*"feel" + 0.015*"people" + 0.01

[(0,
  '0.058*"fuck" + 0.040*"play" + 0.034*"brain" + 0.033*"movie" + 0.032*"remember" + 0.024*"shake" + 0.020*"time" + 0.019*"happen" + 0.019*"girl" + 0.016*"early"'),
 (1,
  '0.065*"work" + 0.029*"house" + 0.028*"tell" + 0.022*"mean" + 0.021*"shit" + 0.021*"stop" + 0.020*"kid" + 0.016*"eye" + 0.015*"way" + 0.014*"live"'),
 (2,
  '0.033*"love" + 0.027*"wake" + 0.025*"think" + 0.024*"face" + 0.021*"write" + 0.021*"dog" + 0.020*"yesterday" + 0.017*"read" + 0.016*"instead" + 0.016*"start"'),
 (3,
  '0.040*"mask" + 0.037*"school" + 0.028*"people" + 0.023*"spider" + 0.021*"attack" + 0.020*"wear" + 0.019*"panic" + 0.017*"forget" + 0.017*"point" + 0.016*"week"'),
 (4,
  '0.025*"turn" + 0.025*"hate" + 0.021*"home" + 0.018*"ass" + 0.018*"ing" + 0.018*"lot" + 0.018*"feel" + 0.015*"people" + 0.015*"kill" + 0.015*"family"')]

In [251]:
## Recc

In [427]:
nmf2=PCA(n_components=5)
nmf2.fit(df_X2)

PCA(n_components=5)

In [439]:
df_X2

,absolutely,abt,abuse,accident,accidentally,account,actually,affect,air,alarm,...,worry,wow,write,yeah,yell,yes,yesterday,zombie,zombie apocalypse,zoom
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4226,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4227,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [429]:
df

,index,content,original,content_clean
0,0,last i had a about a chain chomp from super ...,Last Night I had a nightmare about a chain cho...,"[chain, chomp, super, mario, not, sweat, wth, ..."
1,3,corona got me so nervous i had a that i was a...,"Corona got me so nervous, I had a nightmare th...","[corona, get, gathering, mask, suddenly, start..."
2,4,ive been trading so much past few i i had a ...,"I’ve been trading so much past few days, I thi...","[trade, buy, card, price]"
3,7,i had a about work last in the i told a cus...,I had a dream about work last night. In the dr...,"[work, tell, customer, not]"
4,8,cant believe i had a long elaborate about ing...,can’t believe I had a long elaborate nightmare...,"[not, believe, elaborate, ing, adhd, med, refill]"
...,...,...,...,...
4226,6154,ws ing vinny and there was a spider in ganbree...,"ws watching vinny and there was a ""spider"" in ...","[ing, vinny, spider, ganbreeder, skin, feel, n..."
4227,6156,genuinely up in cold sweat because i had a a...,genuinely woke up in cold sweat because i had ...,"[genuinely, sweat, tank, lmfaooooooooooooooo]"
4228,6160,last i had a terrifying that i was being att...,Last night I had a terrifying nightmare that I...,"[terrifying, attack, unicorn, not, scream, fig..."
4229,6164,chucky because i had a about him when i was...,@puccigucci666 chucky because I had a terrible...,"[vividly, movie, not]"


In [430]:
new_coords=nmf2.transform(df_X2)

In [431]:
pd.DataFrame(new_coords,index=df.original)

,0,1,2,3,4
original,,,,,
Last Night I had a nightmare about a chain chomp from super mario 64 and I dont know why but I woke up sweating and I was like wth its just a chain chomp\nMario 64 personalised has ruined me,-0.010459,-0.057198,-0.038525,0.000884,-0.098754
"Corona got me so nervous, I had a nightmare that I was at a gathering,\nNO MASK, and suddenly started panicking about how many ppl I was around. 😭😭 randomly @BombshellChanel you were there too! Mask less dab!!!! 😂😂",-0.162384,0.292019,1.482935,0.100642,-0.539308
"I’ve been trading so much past few days, I think I had a nightmare about buying a card for 100k over the actual price 😭",-0.003719,-0.051476,-0.048757,-0.012062,-0.095203
I had a dream about work last night. In the dream I told a customer off. This wasn't a nightmare.,-0.072162,0.909981,-0.261918,-0.049100,0.097356
can’t believe I had a long elaborate nightmare about trying to get my ADHD meds refilled 🙃,-0.015963,-0.029197,-0.059451,-0.019955,-0.082646
...,...,...,...,...,...
"ws watching vinny and there was a ""spider"" in ganbreeder and my skin felt so bad :(((( today has not been good bc i had a nightmare this morning i dont rlly wanna talk about but pokemon black has been rlly good..",-0.034487,-0.063453,-0.042532,-0.157682,-0.000993
genuinely woke up in cold sweat because i had a nightmare about tanking LMFAOOOOOOOOOOOOOOO,-0.007419,-0.056122,-0.030385,0.012032,-0.099631
Last night I had a terrifying nightmare that I was being attacked by unicorns (vicious! Not cute! Anti-gay!) and I woke up screaming and fighting the sheets. Glad to see my subconscious is stressing about the right things right now.,-0.063651,0.053262,-0.018329,-0.095539,-0.010062


In [451]:
df.original[101]

'i had a nightmare about returning to middle school 😭😭 stop-'

In [432]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity


In [447]:
def top_3_similar(index):
    dist=pairwise_distances(new_coords[index].reshape(1,-1),new_coords,metric='cosine')
    indices=(dist.argsort().tolist()[0][0:7])
    print ('Tweet: '+ df.original[index])
    print('Recommended Tweets:')
    print('---------------')
    for i in indices:
        print(i,df.original[i])
        print('---------------')
   

In [483]:
df.original[39]

'I had a nightmare that I was stalked by a psychopath because I said something bad about The Grateful Dead.'

In [445]:
top_3_similar(1)

Tweet: Corona got me so nervous, I had a nightmare that I was at a gathering,
NO MASK, and suddenly started panicking about how many ppl I was around. 😭😭 randomly @BombshellChanel you were there too! Mask less dab!!!! 😂😂
Recommended Tweets:
---------------
1 Corona got me so nervous, I had a nightmare that I was at a gathering,
NO MASK, and suddenly started panicking about how many ppl I was around. 😭😭 randomly @BombshellChanel you were there too! Mask less dab!!!! 😂😂
---------------
4126 I had a dream I went to a store without a mask on and I started panicking because I was worried about myself and everyone else there.

Usually, I can will my dreams to shift but I couldn't.

Thanks for the super weirdly specific nightmare, 2020
---------------
1461 So last night it finally happened. I had a nightmare about being in a crowded place and I couldn’t find my mask. I started having trouble breathing and I didn’t know if it was #TheRona or I was be suffocated by the crowd 🥴
---------------
4

In [462]:
df.content_clean[1]

['corona',
 'get',
 'gathering',
 'mask',
 'suddenly',
 'start',
 'panic',
 'ppl',
 'randomly',
 'mask',
 'dab']

In [448]:
top_3_similar(4126)

Tweet: I had a dream I went to a store without a mask on and I started panicking because I was worried about myself and everyone else there.

Usually, I can will my dreams to shift but I couldn't.

Thanks for the super weirdly specific nightmare, 2020
Recommended Tweets:
---------------
4126 I had a dream I went to a store without a mask on and I started panicking because I was worried about myself and everyone else there.

Usually, I can will my dreams to shift but I couldn't.

Thanks for the super weirdly specific nightmare, 2020
---------------
1 Corona got me so nervous, I had a nightmare that I was at a gathering,
NO MASK, and suddenly started panicking about how many ppl I was around. 😭😭 randomly @BombshellChanel you were there too! Mask less dab!!!! 😂😂
---------------
1461 So last night it finally happened. I had a nightmare about being in a crowded place and I couldn’t find my mask. I started having trouble breathing and I didn’t know if it was #TheRona or I was be suffocated b

In [435]:
top_3_similar(1122)

Tweet: I had a nightmare last night. 😭😭😭😭😭

It was about me not doing my assignment padahal dah habis belajar.
Recommended Tweets:
---------------
I had a nightmare last night. 😭😭😭😭😭

It was about me not doing my assignment padahal dah habis belajar.
---------------
I had a nightmare about my assignments.........
---------------
Tell me why I had a nightmare about having two English assignments due when I’ve been out of uni for TWO YEARS 😭
---------------
@b3rryb00 i had a nightmare about it last night 😞😞 fuckin sucked bruh
---------------


In [436]:
top_3_similar(20)

Tweet: I had a terrifying nightmare once, whilst I was at uni, about a particular mythological character - yes, I am a weird Classicist with an over-active imagination - and anyway, she's been brewing in my head for a few years. 

I think she just decided to put herself into words!
Recommended Tweets:
---------------
I had a terrifying nightmare once, whilst I was at uni, about a particular mythological character - yes, I am a weird Classicist with an over-active imagination - and anyway, she's been brewing in my head for a few years. 

I think she just decided to put herself into words!
---------------
@hellpupps i had a nightmare just last night about the scene from who framed roger rabbit where the villain gets flattened and then stands up and re-inflates himself and his cartoon eyes pop out of his head
---------------
correction: I had a nightmare last night about one snake what that mean?
---------------
I had a nightmare about a movie I watched lmfaooo I never woke up like wtf wa

df.content[1122]

In [437]:
for i in range(0,len(df.content)):
    if 'trump' in df.content[i]:
        print (i,df.content[i])


234 oh fuck i had a  that trump was still running to be elected and he called up paul ryan to the stage to do a rap about tanks and flags dear god my subconscious knows how to fuck with me
542 been having  for the last  months about trump last  i had a  i had to go to court for celebrating trumps lost and the judge asked me how i celebrated i  i went cow tipping i  up laughing wonder what that means  tipping 
716 i had a  about tiffany trump and matt gaetz
719 i had a  that the government decided to give trump a second term and didnt care about our votes
770 i had a  about the election they did a recount and trump ended up winning by a lot (on some scam shit of course) it was  the oscars when they announced la la land as the winner but it was really moonlight 
861 if anyone  to rant about the presidential campaign if u follow me and supports that trumpet of a head unfollow me now  i had a full  yr  thanks to him
893 i had a  about the election too a few   and it was a  where trump won 

In [438]:
top_3_similar(3491)

Tweet: I had a dream the other day that @BarackObama woke me up. I said something about Trump being president and he replied "what a nightmare. " and then I woke up for real.
Recommended Tweets:
---------------
I had a dream the other day that @BarackObama woke me up. I said something about Trump being president and he replied "what a nightmare. " and then I woke up for real.
---------------
I had a nightmare about Donald Trump and I’m not even American
---------------
I had a nightmare about Tiffany Trump and Matt Gaetz.
---------------
I had a NIGHTMARE about trump being re-elected
---------------


In [453]:
top_3_similar(2636)

Tweet: last night I had a nightmare that I forgot to wear my mask and started to cry about it on the train while everyone coughed on me.
Recommended Tweets:
---------------
2636 last night I had a nightmare that I forgot to wear my mask and started to cry about it on the train while everyone coughed on me.
---------------
211 I had a nightmare I was eating in a crowded restaurant with no masks and T*ump showed up and was lecturing everyone about how great he is or whatever. STRESS. But I woke up and saw Adam had gone out and got donuts so overall today's off to a good start.
---------------
384 I shit you not, last night I had a "going to class with no pants on" style of nightmare but it was about going to the store and forgetting to wear my mask 😳😳😳
---------------
4126 I had a dream I went to a store without a mask on and I started panicking because I was worried about myself and everyone else there.

Usually, I can will my dreams to shift but I couldn't.

Thanks for the super weirdl

In [459]:
top_3_similar(1108)

Tweet: I had a nightmare about a homework assignment from highschool. Its been like 10 years and I still can't escape school dreams.
Recommended Tweets:
---------------
1108 I had a nightmare about a homework assignment from highschool. Its been like 10 years and I still can't escape school dreams.
---------------
833 Talk about a nightmare, I had a dream last night that I was taking an AP Euro test in high school. 
💀☠️💀☠️
---------------
1907 Of course I had a nightmare about taking a math test in school... https://t.co/xbTPowWbjv
---------------
101 i had a nightmare about returning to middle school 😭😭 stop-
---------------
4068 Can anyone tell me why I had a nightmare about not being able to remember my class schedule from my SOPHOMORE YEAR IN HIGH SCHOOL ???????
---------------
3467 i had a nightmare about school and i’m not even back to classes yet 😐
---------------
4003 Lmao of course I had a nightmare about school on the eve of the first day lolol
---------------


In [487]:
doc_word = tv.fit_transform(docs_list_clean).transpose()

In [488]:
pd.DataFrame(doc_word.toarray(), tv.get_feature_names())

,0,1,2,3,4,5,6,7,8,9,...,4221,4222,4223,4224,4225,4226,4227,4228,4229,4230
absolutely,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
abuse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accident,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
accidentally,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
yell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yesterday,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zombie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
